# Install library

In [1]:
!pip install panda lxml bs4 httpx backtesting yfinance pandas_ta scipy rich TA-Lib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.1/357.1 kB 7.9 MB/s eta 0:00:0000:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for TA-Lib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [21 lines of output]
      <string>:77: UserWarning: Cannot find ta-lib library, installation may fail.
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.linux-x86_64-cpython-310
      creating build/lib.linux-x86_64-cpython-310/talib
      copying talib/deprecated.py -> build/lib.linux-x86_64-cpython-310/talib
      copying talib/abstract.py -> build/lib.linux-x86_64-cpython-310/talib
      copying talib/stream.py -> build/lib.linux-x86_64-cpython-310/talib
      copying talib/__init__.py -> build/lib.linux-x86_64-cpython-310/t

In [2]:
# import library
import pandas as pd
import numpy as np
from datetime import datetime

# Install library

In [3]:
# import library
import pandas as pd
import numpy as np
from datetime import datetime

In [4]:
import concurrent.futures
from datetime import datetime
import glob
import warnings
from rich.progress import track
warnings.filterwarnings("ignore")
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import pandas_ta as taPanda
import talib
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
from collections import Counter

ModuleNotFoundError: No module named 'talib'

# Utilities Function

In [ ]:
def make_api_request(api_endpoint, params):
    with httpx.Client() as client:
        # Make the GET request to the API
        response = client.get(api_endpoint, params=params)
        if response.status_code == 200:
            return response.json()
        print("Error: Failed to retrieve data from API")
        return None

In [ ]:
BASE_URL_FMP = "https://financialmodelingprep.com/api/v3"
BASE_URL_BINANCE = "https://fapi.binance.com/fapi/v1/"

FMP_API_KEY = ""
BINANCE_API_KEY = ""


def get_historical_price_full_crypto(symbol):
    api_endpoint = f"{BASE_URL_FMP}/historical-price-full/crypto/{symbol}"
    params = {"apikey": FMP_API_KEY}
    return make_api_request(api_endpoint, params)


def get_historical_price_full_stock(symbol):
    api_endpoint = f"{BASE_URL_FMP}/historical-price-full/{symbol}"
    params = {"apikey": FMP_API_KEY}

    return make_api_request(api_endpoint, params)

def get_SP500():
    api_endpoint = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    data = pd.read_html(api_endpoint)
    return list(data[0]['Symbol'])

def get_all_crypto():
    """
    All possible crypto symbols
    """
    return [
        "BTCUSD",
        "ETHUSD",
        "LTCUSD",
        "BCHUSD",
        "XRPUSD",
        "EOSUSD",
        "XLMUSD",
        "TRXUSD",
        "ETCUSD",
        "DASHUSD",
        "ZECUSD",
        "XTZUSD",
        "XMRUSD",
        "ADAUSD",
        "NEOUSD",
        "XEMUSD",
        "VETUSD",
        "DOGEUSD",
        "OMGUSD",
        "ZRXUSD",
        "BATUSD"
        "USDTUSD",
        "LINKUSD",
        "BTTUSD",
        "BNBUSD",
        "ONTUSD",
        "QTUMUSD",
        "ALGOUSD",
        "ZILUSD",
        "ICXUSD",
        "KNCUSD",
        "ZENUSD",
        "THETAUSD",
        "IOSTUSD",
        "ATOMUSD",
        "MKRUSD",
        "COMPUSD",
        "YFIUSD",
        "SUSHIUSD",
        "SNXUSD",
        "UMAUSD",
        "BALUSD",
        "AAVEUSD",
        "UNIUSD",
        "RENBTCUSD",
        "RENUSD",
        "CRVUSD",
        "SXPUSD",
        "KSMUSD",
        "OXTUSD",
        "DGBUSD",
        "LRCUSD",
        "WAVESUSD",
        "NMRUSD",
        "STORJUSD",
        "KAVAUSD",
        "RLCUSD",
        "BANDUSD",
        "SCUSD",
        "ENJUSD",
    ]

def get_financial_statements_lists():
    api_endpoint = f"{BASE_URL_FMP}/financial-statement-symbol-lists"
    params = {"apikey": FMP_API_KEY}
    return make_api_request(api_endpoint, params)

# Split the Data into Train and Test

In [ ]:
stock_symbol = "AAPL"
stock_prices = get_historical_stock_prices(stock_symbol)

january_2023_index = stock_prices.index("2023-01-01") if "2023-01-01" in stock_prices else None

if january_2023_index is not None:
    prices_before_january_2023 = stock_prices[:january_2023_index]
    prices_after_january_2023 = stock_prices[january_2023_index:]
else:
    print("January 2023 data not found in the historical prices.")

# Monte Carlo Simulation

In [12]:
def monte_carlo_simulation(data, days, iterations):
    log_returns = np.log(data[1:] / data[:-1])
    mean = np.mean(log_returns)
    variance = np.var(log_returns)
    drift = mean - (0.5 * variance)
    daily_volatility = np.std(log_returns)

    future_prices = np.zeros((days, iterations))
    current_price = data[-1]
    for t in range(days):
        shocks = drift + daily_volatility * norm.ppf(np.random.rand(iterations))
        future_prices[t] = current_price * np.exp(shocks)
        current_price = future_prices[t]
    return future_prices

# Markov Chain Simulation Transition Matrix

In [ ]:
# Markov Chain Simulation (Simplified)
transition_matrix = np.array([[0.7, 0.2, 0.1],  # Transition probabilities for states
                              [0.1, 0.6, 0.3],
                              [0.2, 0.3, 0.5]])

# Markov Chain Simulation

In [ ]:
def markov_chain_simulation(transition_matrix, start_state, days):
    states = np.arange(len(transition_matrix))
    current_state = start_state
    state_sequence = [current_state]

    for _ in range(days):
        current_state = np.random.choice(states, p=transition_matrix[current_state])
        state_sequence.append(current_state)

    return state_sequence

#  MCMCStrategy

In [ ]:
class MCMCStrategy(Strategy):
    def init(self):
        self.is_long = False
        simulation_days = len(self.data)
        mc_iterations = 100
        start_state = 0  # Initial state (can be changed)
        self.mc_prices = monte_carlo_simulation(self.data.Close, len(self.data), mc_iterations)
        self.markov_states = markov_chain_simulation(transition_matrix, start_state, len(self.data))

    def next(self):
        # MCMC Strategy
        if self.is_long:
            if self.data.Close[-1] < self.mc_prices[-1, self.i]:
                self.position.close()
                self.is_long = False
        else:
            if self.data.Close[-1] > self.mc_prices[-1, self.i]:
                self.buy()
                self.is_long = True

# Create two separate Backtest instances for before and after January 2023

In [ ]:

bt_mcmc_before_january = Backtest(prices_before_january_2023, MCMCStrategy,
                             cash=10000, commission=.002,
                             exclusive_orders=True)

bt_mcmc_after_january = Backtest(prices_after_january_2023, MCMCStrategy,
                            cash=10000, commission=.002,
                            exclusive_orders=True)

In [ ]:
output_mcmc_before_january = bt_before_january.run()

In [ ]:
output_mcmc_after_january = bt_after_january.run()

# Run BUYHOLD backtests

In [ ]:
class BuyAndHoldStrategy(Strategy):
    def init(self):
        self.is_long = False

    def next(self):
        # Buy and Hold Strategy
        if not self.is_long:
            self.buy()
            self.is_long = True

In [ ]:

bt_before_january = Backtest(prices_before_january_2023, BuyAndHoldStrategy, cash=10000, commission=.002, exclusive_orders=True)
bt_after_january = Backtest(prices_after_january_2023, BuyAndHoldStrategy, cash=10000, commission=.002, exclusive_orders=True)

In [ ]:
before_after_january = bt_after_january.run()

In [ ]:
output_after_january = bt_after_january.run()

## Access performance metrics for BUYHOLD backtests

In [ ]:

print("Performance Metrics Before January 2023:")
print(output_before_january)
print("\nPerformance Metrics After January 2023:")
print(output_after_january)

In [ ]:
# Define a function to run a backtest and plot the results
def run_backtest_and_plot(prices, strategy_class, title):
    bt = Backtest(prices, strategy_class, cash=10000, commission=.002, exclusive_orders=True)
    output = bt.run()
    bt.plot(title=title)
    return output

In [ ]:
def buy_and_hold_backtest(prices):
    simulation_days = len(prices)
    mc_iterations = 100
    start_state = 0  # Initial state (can be changed)
    mc_prices = monte_carlo_simulation(prices, simulation_days, mc_iterations)
    markov_states = markov_chain_simulation(transition_matrix, start_state, simulation_days)

    # Simple "Buy and Hold" Strategy Backtest
    final_prices = mc_prices[-1, :]  # Prices at the end of the simulation
    initial_price = prices[-1]  # Initial price
    portfolio_values = final_prices - initial_price  # Portfolio values

    # Calculate strategy returns
    strategy_returns = portfolio_values / initial_price
    return strategy_returns

def run_backtest_and_plot(prices, strategy_class, title):
    bt = Backtest(prices, strategy_class, cash=10000, commission=.002, exclusive_orders=True)
    output = bt.run()
    bt.plot(title=title)
    return output

# Perform "Buy and Hold" backtest for prices before January 2023
strategy_returns_before_january = buy_and_hold_backtest(prices_before_january_2023)

# Perform "Buy and Hold" backtest for prices after January 2023
strategy_returns_after_january = buy_and_hold_backtest(prices_after_january_2023)

# Visualize the results
plt.figure(figsize=(10, 6))
plt.plot(strategy_returns_before_january, label="Before January 2023")
plt.plot(strategy_returns_after_january, label="After January 2023")
plt.title('Buy and Hold Strategy Returns')
plt.xlabel('Simulation Iterations')
plt.ylabel('Returns')
plt.legend()
plt.show()

# Run and plot the MCMC-based backtests
output_before_january_mcmc = run_backtest_and_plot(prices_before_january_2023, MCMCStrategy, "MCMC Strategy Before January 2023")
output_after_january_mcmc = run_backtest_and_plot(prices_after_january_2023, MCMCStrategy, "MCMC Strategy After January 2023")

# Access performance metrics for both MCMC-based backtests
print("Performance Metrics for MCMC Strategy Before January 2023:")
print(output_before_january_mcmc)
print("\nPerformance Metrics for MCMC Strategy After January 2023:")
print(output_after_january_mcmc)